In [23]:
# -*- coding: utf-8 -*-
import os
import re
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from pathlib import Path
from difflib import get_close_matches

import geopandas as gpd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import joblib
import matplotlib.pyplot as plt
import seaborn as sns


In [24]:
DATA_DIR = Path('.')
crop_csv = DATA_DIR / 'crop_data.csv'             # Crop data CSV
gadm_geojson = DATA_DIR / 'gadm41_IND_2.json'    # District-level GeoJSON
print(gadm_geojson.exists())
# Load crop data
df = pd.read_csv(crop_csv)
print("Crop data shape:", df.shape)
display(df.head())

gadm = None
if gpd is not None and gadm_geojson.exists():
    gadm = gpd.read_file(gadm_geojson)
    print("GADM GeoJSON loaded. GADM shape:", gadm.shape)
    display(gadm.head())



True
Crop data shape: (50765, 20)


,Dist Code,Year,State Code,State Name,Dist Name,Crop,Area_ha,Yield_kg_per_ha,N_req_kg_per_ha,P_req_kg_per_ha,K_req_kg_per_ha,Total_N_kg,Total_P_kg,Total_K_kg,Temperature_C,Humidity_%,pH,Rainfall_mm,Wind_Speed_m_s,Solar_Radiation_MJ_m2_day
0,1,1966,14,Chhattisgarh,Durg,rice,548000.0,337.59,8.43975,4.05108,7.42698,4624983.00,2219991.84,4069985.04,25,80,6.5,1200,2.0,18
1,1,1966,14,Chhattisgarh,Durg,maize,3000.0,666.67,18.00009,8.00004,11.33339,54000.27,24000.12,34000.17,22,70,6.0,800,2.5,20
2,1,1966,14,Chhattisgarh,Durg,chickpea,54000.0,500.00,9.00000,5.00000,9.00000,486000.00,270000.00,486000.00,20,60,6.5,600,1.5,16
3,1,1967,14,Chhattisgarh,Durg,rice,547000.0,747.71,18.69275,8.97252,16.44962,10224934.25,4907968.44,8997942.14,25,80,6.5,1200,2.0,18
4,1,1967,14,Chhattisgarh,Durg,maize,3000.0,1000.00,27.00000,12.00000,17.00000,81000.00,36000.00,51000.00,22,70,6.0,800,2.5,20


GADM GeoJSON loaded. GADM shape: (676, 14)


,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,IND.1.1_1,IND,India,IND.1_1,AndamanandNicobar,NA,NicobarIslands,NA,NA,District,District,NA,IN.AN.NI,"MULTIPOLYGON (((93.7899 6.852, 93.7909 6.851, ..."
1,IND.1.2_1,IND,India,IND.1_1,AndamanandNicobar,NA,NorthandMiddleAndaman,NA,NA,District,District,NA,IN.AN.NM,"MULTIPOLYGON (((92.8444 12.1497, 92.8461 12.15..."
2,IND.1.3_1,IND,India,IND.1_1,AndamanandNicobar,NA,SouthAndaman,NA,NA,District,District,NA,IN.AN.SA,"MULTIPOLYGON (((92.5211 10.8969, 92.53 10.8869..."
3,IND.2.1_1,IND,India,IND.2_1,AndhraPradesh,NA,Anantapur,"Anantpur,Ananthapur",NA,District,District,NA,IN.AD.AN,"MULTIPOLYGON (((77.846 13.9283, 77.8301 13.927..."
4,IND.2.2_1,IND,India,IND.2_1,AndhraPradesh,NA,Chittoor,Chitoor|Chittor,NA,District,District,NA,IN.AD.CH,"MULTIPOLYGON (((78.5455 12.7439, 78.5503 12.73..."


In [25]:
def normalize_name(s):
    if pd.isna(s): return ""
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Normalize state & district names
df['state_norm'] = df['State Name'].astype(str).apply(normalize_name)
df['dist_norm']  = df['Dist Name'].astype(str).apply(normalize_name)

if gadm is not None:
    gadm['state_norm'] = gadm['NAME_1'].astype(str).apply(normalize_name)
    gadm['dist_norm']  = gadm['NAME_2'].astype(str).apply(normalize_name)


In [39]:
# if gadm is not None:
#     merged = df.merge(gadm[['state_norm','dist_norm']], on=['state_norm','dist_norm'], how='left', indicator=True)
#     unmatched = merged[merged['_merge']=='left_only'].copy()
#     print("Exact merge unmatched rows:", len(unmatched))

#     # Fuzzy matching for unmatched rows
#     if len(unmatched) > 0:
#         state_to_dists = {s: set(dists) for s, dists in zip(gadm['state_norm'], gadm['dist_norm'])}
#         all_dists = gadm['dist_norm'].unique().tolist()

#         def find_best_dist(state_norm, dist_norm, cutoff=0.7):
#             candidates = list(state_to_dists.get(state_norm, []))
#             matches = get_close_matches(dist_norm, candidates, n=1, cutoff=cutoff) or get_close_matches(dist_norm, all_dists, n=1, cutoff=cutoff)
#             return matches[0] if matches else None

#         for idx, row in unmatched.iterrows():
#             matched_d = find_best_dist(row['state_norm'], row['dist_norm'], cutoff=0.72)
#             if matched_d:
#                 merged.at[idx, 'dist_norm'] = matched_d

#         merged = merged.drop(columns=[c for c in merged.columns if c.startswith('geometry')], errors='ignore')
#         merged = merged.drop(columns=['_merge'], errors='ignore')
#         merged = merged.merge(gadm[['state_norm','dist_norm']], on=['state_norm','dist_norm'], how='left', indicator=True)
#         print("After fuzzy re-merge unmatched rows:", (merged['_merge']=='left_only').sum())
# else:
#     merged = df.copy()
#     print("GADM not available; skipping spatial merge.")

# df_work = merged.copy()
# print("Working dataframe shape:", df_work.shape)


Exact merge unmatched rows: 30809
After fuzzy re-merge unmatched rows: 25438
Working dataframe shape: (51198, 23)


In [40]:
# Numeric columns
num_cols = [
    'Area_ha','Yield_kg_per_ha',
    'N_req_kg_per_ha','P_req_kg_per_ha','K_req_kg_per_ha',
    'Total_N_kg','Total_P_kg','Total_K_kg',
    'Temperature_C','Humidity_%','pH','Rainfall_mm','Wind_Speed_m_s','Solar_Radiation_MJ_m2_day'
]

for c in num_cols:
    if c in df_work.columns:
        df_work[c] = pd.to_numeric(df_work[c].astype(str).str.replace(',','').replace(['nan','None',''], np.nan), errors='coerce')

# Avoid division by zero
df_work.loc[df_work['Area_ha']==0, 'Area_ha'] = np.nan

# Fertilizer application per hectare
df_work['applied_N_per_ha'] = df_work['Total_N_kg'] / df_work['Area_ha']
df_work['applied_P_per_ha'] = df_work['Total_P_kg'] / df_work['Area_ha']
df_work['applied_K_per_ha'] = df_work['Total_K_kg'] / df_work['Area_ha']

# Deficits and fraction of requirement
for nutrient in ['N','P','K']:
    df_work[f'{nutrient}_deficit_kg_per_ha'] = df_work[f'applied_{nutrient}_per_ha'] - df_work[f'{nutrient}_req_kg_per_ha']
    df_work[f'{nutrient}_frac_of_req'] = df_work[f'applied_{nutrient}_per_ha'] / (df_work[f'{nutrient}_req_kg_per_ha'] + 1e-9)

# Crop -> Season
KHARIF = {'rice','maize','cotton','soybean','groundnut','bajra','jowar','sorghum'}
RABI = {'wheat','barley','gram','mustard','linseed','pea','rapeseed'}
ZAID = {'watermelon','muskmelon','cucumber','vegetables','fodder'}

def crop_to_season(crop):
    if pd.isna(crop): return 'unknown'
    s = str(crop).strip().lower()
    if s in KHARIF: return 'Kharif'
    if s in RABI: return 'Rabi'
    if s in ZAID: return 'Zaid'
    return 'Other'

df_work['Season'] = df_work['Crop'].apply(crop_to_season)


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50765 entries, 0 to 50764
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Dist Code                  50765 non-null  int64  
 1   Year                       50765 non-null  int64  
 2   State Code                 50765 non-null  int64  
 3   State Name                 50765 non-null  object 
 4   Dist Name                  50765 non-null  object 
 5   Crop                       50765 non-null  object 
 6   Area_ha                    50765 non-null  float64
 7   Yield_kg_per_ha            50765 non-null  float64
 8   N_req_kg_per_ha            50765 non-null  float64
 9   P_req_kg_per_ha            50765 non-null  float64
 10  K_req_kg_per_ha            50765 non-null  float64
 11  Total_N_kg                 50765 non-null  float64
 12  Total_P_kg                 50765 non-null  float64
 13  Total_K_kg                 50765 non-null  flo

In [30]:
df_model = df_work.dropna(subset=['Yield_kg_per_ha']).copy()

numeric_features = [c for c in [
    'Area_ha',
    'applied_N_per_ha','applied_P_per_ha','applied_K_per_ha',
    'N_req_kg_per_ha','P_req_kg_per_ha','K_req_kg_per_ha',
    'N_deficit_kg_per_ha','P_deficit_kg_per_ha','K_deficit_kg_per_ha',
    'N_frac_of_req','P_frac_of_req','K_frac_of_req',
    'Temperature_C','Humidity_%','pH','Rainfall_mm','Wind_Speed_m_s','Solar_Radiation_MJ_m2_day','Year'
] if c in df_model.columns]

categorical_features = [c for c in ['State Name','Dist Name','Crop','Season'] if c in df_model.columns]

X = df_model[numeric_features + categorical_features]
y = df_model['Yield_kg_per_ha'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [31]:
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Use XGBoost if available
try:
    model = XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, objective='reg:squarederror', n_jobs=-1, random_state=42)
    print("Using XGBoost")
except:
    model = RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)
    print("Using RandomForest")

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', model)
])

pipeline.fit(X_train, y_train)


Using XGBoost


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Area_ha',
                                                   'applied_N_per_ha',
                                                   'applied_P_per_ha',
                                                   'applied_K_per_ha',
                                                   'N_req_kg_per_ha',
                                                   'P_req_kg_per_ha',
                                                   'K_req_kg_per_ha',
                                                   'N_deficit_kg_per_ha',
                                                   'P_deficit_kg_per_ha',
                                                   'K_deficit_kg_per_ha',...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=6, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=300, n_jobs=-1,
                              num_parallel_tree=None, ...))])

In [32]:
from math import sqrt

y_pred = pipeline.predict(X_test)
print(f"MAE: {mean_absolute_error(y_test, y_pred):.3f}")
print(f"RMSE: {sqrt(mean_squared_error(y_test, y_pred)):.3f}")
print(f"R2: {r2_score(y_test, y_pred):.3f}")


MAE: 4.029
RMSE: 13.050
R2: 0.999


In [33]:
def get_feature_names(preprocessor):
    num_names = numeric_features
    cat_names = list(preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features))
    return num_names + cat_names

feat_names = get_feature_names(pipeline.named_steps['preprocessor'])
reg = pipeline.named_steps['regressor']
importances = reg.feature_importances_
imp_df = pd.DataFrame({'feature': feat_names, 'importance': importances}).sort_values('importance', ascending=False)
display(imp_df.head(30))


,feature,importance
11,P_frac_of_req,0.363142
5,P_req_kg_per_ha,0.352982
2,applied_P_per_ha,0.242409
10,N_frac_of_req,0.018697
1,applied_N_per_ha,0.005980
13,Temperature_C,0.005546
3,applied_K_per_ha,0.002403
14,Humidity_%,0.001880
40,Dist Name_Khargone / West Nimar,0.001202
32,Dist Name_Dhar,0.001121


In [34]:
joblib.dump(pipeline, DATA_DIR/'crop_yield_pipeline.joblib')
df_model.to_csv(DATA_DIR/'crop_data_merged.csv', index=False)


In [35]:
def apply_suggestions(row):
    """Apply agronomic suggestions for improvement"""
    improved = row.copy()
    # Balance nutrients
    for nutrient in ['N','P','K']:
        improved[f'applied_{nutrient}_per_ha'] = max(improved[f'applied_{nutrient}_per_ha'], improved[f'{nutrient}_req_kg_per_ha'])
    # Soil pH
    improved['pH'] = min(max(row['pH'], 6.5), 7.5) if not pd.isna(row['pH']) else 6.8
    # Weather adjustments
    improved['Rainfall_mm'] = max(row.get('Rainfall_mm',0), 400)
    improved['Temperature_C'] = min(max(row.get('Temperature_C',0), 20), 30)
    return improved

def generate_recommendations(row, pipeline):
    recs = []
    for nutrient in ['N','P','K']:
        if row[f'{nutrient}_req_kg_per_ha'] > row[f'applied_{nutrient}_per_ha']*1.05:
            recs.append(f"{nutrient} deficit → Add fertilizer")
    if row['pH'] < 5.5: recs.append("Soil acidic → Apply lime")
    if row['pH'] > 8.0: recs.append("Soil alkaline → Apply gypsum/organic amendments")
    improved_row = apply_suggestions(row)
    base_yield = pipeline.predict(pd.DataFrame([row]))[0]
    improved_yield = pipeline.predict(pd.DataFrame([improved_row]))[0]
    recs.append(f"Predicted yield: {base_yield:.2f} kg/ha")
    recs.append(f"Predicted yield with suggestions: {improved_yield:.2f} kg/ha (+{((improved_yield-base_yield)/base_yield*100):.1f}%)")
    return recs


In [36]:
{
  "Area_ha": 1.5,
  "Crop": "Rice",
  "State Name": "Karnataka",
  "Dist Name": "Bangalore Rural",
  "N_req_kg_per_ha": 50,
  "P_req_kg_per_ha": 25,
  "K_req_kg_per_ha": 20,
  "applied_N_per_ha": 30,
  "applied_P_per_ha": 10,
  "applied_K_per_ha": 5,
  "Temperature_C": 32,
  "Humidity_%": 70,
  "pH": 5.5,
  "Rainfall_mm": 300,
  "Wind_Speed_m_s": 2.5,
  "Solar_Radiation_MJ_m2_day": 18
}


{'Area_ha': 1.5,
 'Crop': 'Rice',
 'State Name': 'Karnataka',
 'Dist Name': 'Bangalore Rural',
 'N_req_kg_per_ha': 50,
 'P_req_kg_per_ha': 25,
 'K_req_kg_per_ha': 20,
 'applied_N_per_ha': 30,
 'applied_P_per_ha': 10,
 'applied_K_per_ha': 5,
 'Temperature_C': 32,
 'Humidity_%': 70,
 'pH': 5.5,
 'Rainfall_mm': 300,
 'Wind_Speed_m_s': 2.5,
 'Solar_Radiation_MJ_m2_day': 18}